In [0]:
df_silver  = spark.table('data_engineering_project.silver_orders')
df_silver.display()

Creating calendar Dimension 

In [0]:
%sql
CREATE TABLE IF NOT EXISTS data_engineering_project.gold_dim_date (
  date_key INT,                 
  full_date DATE,
  day INT,
  day_name STRING,
  day_of_week INT,
  week_of_year INT,
  month INT,
  month_name STRING,
  quarter INT,
  year INT,
  is_weekend BOOLEAN
)
USING DELTA;


Populating the table

In [0]:
spark.sql(f'''
          MERGE INTO data_engineering_project.gold_dim_date t 
          USING(
            WITH maxnmin AS (
              SELECT 
                LEAST(min(order_date),min(ship_date)) AS start_date,
                GREATEST(max(order_date),max(ship_date)) As end_date
                FROM data_engineering_project.silver_orders
            ),
            dates AS (
              SELECT explode(
                sequence(start_date, end_date, interval 1 day)
              )AS full_date
              FROM maxnmin
              )

              SELECT 
                ROW_NUMBER() OVER (ORDER BY full_date) AS date_key,
                full_date, 
                day(full_date) AS day, 
                date_format(full_date,'EEEE') As day_name, 
                dayofweek(full_date) As day_of_week,
                weekofyear(full_date) AS week_of_year,
                month(full_date) AS month,
                date_format(full_date,'MMMM') AS month_name,
                quarter(full_date) AS quarter,
                year(full_date) AS year,
                CASE 
                    WHEN dayofweek(full_date) IN (1,7) THEN True
                    ELSE False
                END AS is_weekend
                FROM dates
          ) AS s 
          ON t.full_date = s.full_date
          WHEN NOT MATCHED THEN INSERT *;
          ''')


In [0]:
%sql
SELECT * FROM data_engineering_project.gold_dim_date